# 2장. 단일 책임 원칙을 따르는 클래스 디자인하기

객체지향 시스템은 클래스들 사이에 주고 받는 메세지로 이루어진다

클래스를 만들기 위해서는(20 p)
1. 연관성 있는 메서드들을 포함하도록
2. 수정하기 쉽도록

수정하기 쉬운 코드의 특징(21 p)
1. 투명하다
2. 적절하다
3. 사용가능하다
4. 모범이 된다

### 하나의 책임만 가지는 클래스를 만들어보자

### Page 22

In [ ]:
chainring = 52  # 앞 톱니 개수
cog = 11  # 뒷 톱니 개수
ratio = chainring / float(cog)  # 기어비
print(ratio)

chainring = 30
cog = 27
ratio = chainring / float(cog)
print(ratio)

놀랍도록 기어비에 신경쓰는 사람들이 있습니다. 그게 저는 아닙니다만 
http://www.bikem.co.kr/article/read.php?num=8519

### Page 23

In [ ]:
class Gear(object):
    def __init__(self, chainring, cog):
        """
        앞, 뒤 톱니바퀴수
        """
        self.__chainring = chainring
        self.__cog = cog

    def ratio(self):
        return self.__chainring / float(self.__cog)

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

In [ ]:
def test_gear_ratio():
    print(Gear(52, 11).ratio())
    print(Gear(30, 27).ratio())

In [ ]:
test_gear_ratio()

아무 문제 없어 보이지만, 바퀴의 크기가 다른 경우가 있다. 예를 들면 이런 경우?  
26인치 vs 29인치 : http://m.blog.naver.com/searider/220055184841  
폴딩 : http://blog.naver.com/PostView.nhn?blogId=ninjarazzi&logNo=40117828380

새로운 행동을 추가하자

### Page 20

In [ ]:
class Gear(object):
    def __init__(self, chainring, cog, rim, tire):
        """
        앞, 뒤 톱니바퀴수
        바퀴테 지름
        타이어 높이
        """
        self.__chainring = chainring
        self.__cog = cog
        self.__rim = rim
        self.__tire = tire

    def ratio(self):
        return self.chainring / float(self.cog)

    def gear_inches(self):
        # 타이어는 바퀴를 감싸고 있으므로, 지름을 계산할 때 타이어 높이에 2를 곱한다
        return self.ratio() * (self.rim + (self.tire * 2))

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

    @property
    def rim(self):
        return self.__rim

    @property
    def tire(self):
        return self.__tire

In [ ]:
print(Gear(52, 11, 26, 1.5).gear_inches())
print(Gear(52, 11, 24, 1.25).gear_inches())

In [ ]:
# 잘 되던게 안되는 이유
test_gear_ratio()

클래스란 쉽게 가져다 쓸 수 있는 코드  
결합도가 높으면 유지보수가 힘들어 진다  
<br>  
행동에 기반한 코드를 만들어보자

### Page 30

In [ ]:
class Gear(object):
    def __init__(self, chainring, cog):
            self.__chainring = chainring
            self.__cog = cog

    def ratio(self):
        return self.__chainring / float(self.__cog)      # 멸망의 길

In [ ]:
test_gear_ratio()

### Page 30

In [ ]:
class Gear(object):
    def __init__(self, chainring, cog):
        self.__chainring = chainring
        self.__cog = cog

    def ratio(self):
        """
        변수를 감싸고 이를 사용하자
        """
        return self.chainring / float(self.cog)

    @property
    def chainring(self):   
        """
        앞 톱니바퀴수가 무엇인지 아는 프로퍼티를 만든다. 
        계산식이 바뀌더라도 이 부분만 바꿔주면 된다(31 p)
        """
        return self.__chainring

    @property
    def cog(self):
        """
        뒷 톱니바퀴수가 무엇인지 아는 프로퍼티를 만든다. 
        """
        return self.__cog

In [ ]:
test_gear_ratio()

### Page 32 복잡한 데이터 구조에 의존하는 것은 더 안좋다

In [ ]:
class ObscuringReferences(object):
    def __init__(self, data):
        self._data = data

    def diameters(self):
        # 0 is rim, 1 is tire
        return [cell[0] + (cell[1] * 2) for cell in self.data]

    @property
    def data(self):
        return self._data

    # ... many other methods that index into the array

In [ ]:
data = [[622, 20], [622, 23], [559, 30], [559, 40]]

In [ ]:
ors = ObscuringReferences(data)

In [ ]:
# 지금은 잘 동작하지만 배열 구조가 바뀌면 클래스가 가지고 있는 데이터 구조를 바꾸는 것이기 때문에 영향도가 크다
ors.diameters()

### Page 33 

In [1]:
from collections import namedtuple

In [13]:
class RevealingReferences(object):
    def __init__(self, data):
        self.wheels = data

    def diameters(self):
        return [wheel.rim + (wheel.tire * 2) for wheel in self.wheels]

    @property
    def wheels(self):
        return self.__wheels

    Wheel = namedtuple('Wheel', ['rim', 'tire'])

    @wheels.setter
    def wheels(self, value):
        print(value)
        self.__wheels = [self.Wheel(cell[0], cell[1]) for cell in value]

In [14]:
data = [(622, 23), (559, 30), (559, 40)]

In [15]:
rrs = RevealingReferences(data)

[(622, 23), (559, 30), (559, 40)]


In [16]:
rrs.diameters()

[668, 619, 639]

In [17]:
rrs.wheels

[Wheel(rim=622, tire=23), Wheel(rim=559, tire=30), Wheel(rim=559, tire=40)]

In [18]:
Wheel = namedtuple('Wheel', ['rim', 'tire'])

In [19]:
data = [Wheel(622, 33), Wheel(559, 40), Wheel(559, 50)]

In [25]:
rrs.wheels(value=(622, 33))

TypeError: 'list' object is not callable

### Page 35

In [39]:
class RevealingReferences(object):
    def __init__(self, data):
        self.wheels = data
    
    # page 35
    # first - iterate over the array
    def diameters(self):
        return [self.diameter(wheel) for wheel in self.wheels]

    # second - calculate diameter of ONE wheel
    def diameter(self, wheel):
        print("{} + {}".format(wheel.rim, (wheel.tire * 2)))
        return wheel.rim + (wheel.tire * 2)
    
    @property
    def wheels(self):
        return self.__wheels

    Wheel = namedtuple('Wheel', ['rim', 'tire'])

    @wheels.setter
    def wheels(self, value):
        self.__wheels = [self.Wheel(cell[0], cell[1]) for cell in value]

In [40]:
rrs = RevealingReferences(data)

In [41]:
rrs.diameter(Wheel(600, 30))

600 + 60


660

### Page 38

In [42]:
class Gear(object):
    def __init__(self, chainring, cog, rim, tire):
        """
        앞, 뒤 톱니개수
        바퀴테 지름
        타이어 높이
        """
        self.__chainring = chainring
        self.__cog = cog
        self.__wheel = self.Wheel(rim, tire)

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

    @property
    def wheel(self):
        return self.__wheel

    def ratio(self):
        return self.chainring / float(self.cog)

    def gear_inches(self):
        return self.ratio() * self.wheel.diameter()
    
    # 장기적인 목표엔 맞지 않지만..
    class Wheel(namedtuple('Wheel', ['rim', 'tire'])):
        def diameter(self):
            return self.rim + (self.tire * 2)

In [43]:
gear = Gear(50, 15, 26, 3)

In [46]:
gear.chainring, gear.cog, gear.ratio(), gear.gear_inches()

(50, 15, 3.3333333333333335, 106.66666666666667)

### Page 39 진짜 마지막

In [47]:
class Gear(object):
    def __init__(self, chainring, cog, wheel=None):
        """
        앞, 뒤 톱니개수
        바퀴
        """
        self.__chainring = chainring
        self.__cog = cog
        self.__wheel = wheel

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

    @property
    def wheel(self):
        return self.__wheel

    def ratio(self):
        return self.chainring / float(self.cog)

    def gear_inches(self):
        return self.ratio() * self.wheel.diameter()


class Wheel(object):
    def __init__(self, rim, tire):
        self.__rim = rim
        self.__tire = tire

    def diameter(self):
        return self.rim + (self.tire * 2)

    def circumference(self):
        import math
        return self.diameter() * math.pi

    @property
    def rim(self):
        return self.__rim

    @property
    def tire(self):
        return self.__tire

In [50]:
wheel = Wheel(26, 1.5)

In [51]:
wheel.diameter(), wheel.circumference(), wheel.rim, wheel.tire

(29.0, 91.106186954104, 26, 1.5)

In [52]:
gear = Gear(50, 16, wheel)

In [55]:
gear.chainring, gear.cog, gear.wheel.__dict__, gear.ratio(), gear.gear_inches()

(50, 16, {'_Wheel__rim': 26, '_Wheel__tire': 1.5}, 3.125, 90.625)